# 7장 순수 함수적 병렬성(Chapter 7. Purely functional parallelism)

# 차례

* 7.1 자료 형식과 함수의 선택 (Choosing data type and functions)
* 7.2 표현의 선택 (Picking a representation)
* 7.3 API의 정련 (Refining the API)
* 7.4 API의 대수 (The algebra of an API)
* 7.5 조합기들을 가장 일반적인 형태로 정련 (Refining combiators to their most general form)
* 7.6 요약 (Summary)

In [ ]:
val outputlist = parMap(iniputList)(f)  // 이런 조합기, 병렬 및 비동기 계산 방식으로 만드는 것이 이번 장의 궁극목표.

* 순수 함수적 라이브러리의 설계 문제에 대한 접근 방식을 배우자.
* 계산의 <font color="red">서술</font>이라는 관심사를 실제 <font color="red">실행</font>이라는 관심사와 분리하자.
* 항상 대수적 추론에 역점을 두고, API를 특정 법칙(law)를 따르는 하나의 대수(algebra)로서 서술할 수 있다.
* 부수 효과를 절대로 허용하지 않는다.

# 7.1 자료 형식과 함수의 선택 (Choosing data type and functions)

* 7.1.1 병렬 계산을 위한 자료 형식 하나 (A data type for parallel computations)
* 7.1.2 병렬 계산의 조합 (Combining parallel computations)
* 7.1.3 명시적 분기 (Explicit forking)

#### 통상적인 방식

In [ ]:
def sum(ints: Seq[Int]) : Int =
    ints.foldLeft(0)((a,b) => a + b)

#### 분할정복 알고리즘을 적용해보자 ( 병렬화할 수 있다)

<img src="figures/list7.1.png" width=600 >    

## 7.1.1 병렬 계산을 위한 자료 형식 하나 (A data type for parallel computations)

#### 병렬 계산의 결과를 담을 컨테이너 형식

In [ ]:
Par[A]

#### Par[A] 자료형에 필요한 함수들

In [ ]:
def unit[A](a: => A) : Par[A]

In [ ]:
def get[A](a: Par[A]): A

#### Par[A] 자료형식을 적용

<img src="figures/list7.2.png" width=600 >  

#### unit는 

* 주어진 인수를 개별적인 스레드(논리적 스레드)에서 즉기 평가할 수도 있고,
* 인수를 그냥 가지고 있다가 get이 호출되면 평가를 시작할 수도 있다.
    - 그러나 지금 예제이서는 병렬성의 이점을 취하기 위해서는 unit가 동시적 평가를 시작한 후 즉시 반환되어야 한다.(why?)

#### unit가 인수들의 평가를 동시에 시작한다면 

* get 호출에서 참조 투명성이 깨질 수 있다.
* 아래와 같은 경우(치환) 살펴보면, 이제는 프로그램이 병렬로 실행되지 않는다.
* unit가 자신의 인수를 즉시 평가하기 시작하면, 그 다음으로 일어나는 일은 get이 그 평가의 완료를 기다리는 것이다. 
* unit 관련, get에서 한정된 부수효과가 존재하게 된다.

In [ ]:
Par.get(Par.unit(sum(l))) + Par.get(Par.unit(sum(r)))

* 이 경우 unit은 그냥 비동기 계산을 나타내는 Par[Int]를 돌려준다. 
* 그런데 Par를 get으로 즉시 넘겨주는 즉시, get의 완료까지 실행이 차단되는 부수 효과가 드러난다.
* get을 호출하지 않거나, 적어도 호출을 최대한 미루어야 한다.
* 즉, 비동기 계산들을 그 완료를 기다리지 않고도 조합할 수 있어야 한다.

#### 동시성 기본수단을 직접 사용하는 것의 문제점

<img src="figures/cap7.2.png" width=600 >     

In [ ]:
class ExecutorSerive {
    def submit[A](a: Callable[A]) Future[A]
}

trait Future[A] {
    def get: A
}

## 7.1.2 병렬 계산의 조합 (Combining parallel computations)

#### get을 호출하지 않고 Par[Int]를 돌려주도록 바꾸자

In [ ]:
def sum(ints: IndexedSeq[Int]) : Par[Int] =
    if (ints.size <= 1)
        Par.unit(ints.headOptoin getOrElse 0)
    else {
        Val (l, r) = ints.splitAt(ints.length/2)
        Par.map2(sum(l), sum(r))(_ + _)   // 두 병렬 계산의 결과를 결합하는 고차 함수이다.
    }

* unit는 이제 게으른 인수여야 하는지 명확하지 않다. (지금 예제에서는 인수를 게으르게 받는다는 것이 별로 이득이 되지 않는다)
* map2는 인수를 게으르게 받아야 할까?

#### map2의 두 인수를 엄격하게 평가되는 경우

### Listing 7.3 sum의 프로그램 추적

In [ ]:
sum(IndexedSeq(1,2,3,4))
map2(
    sum(IndexedSeq(1,2)),
    sum(IndexedSeq(3,4))(_ + _)
map2(
    map2(
        sum(IndexedSeq(1)),
        sum(IndexedSeq(2)))(_ + _)
    sum(IndexedSeq(3,4)))(_ + _)
map2(
    map2(
        unit(1),
        unit(2))(_ + _)
    sum(IndexedSeq(3,4)))(_ + _)
map2(
    map2(
        unit(1),
        unit(2))(_ + _),
    map2(
        sum(IndexedSeq(3)),
        sum(IndexedSeq(4)))(_ + _))(_ + _)
...

#### map2를 엄격하게  유지하되 그 실행이 즉시 시작되지 않게 하면?

In [ ]:
map2(
    map2(
        unit(1),
        unit(2))(_ + _),
    map2(
        unit(3),
        unit(4))(_ + _))(_ + _)

#### map2를 게으르게 만들고 양변을 병렬로 즉시 실행하는게 나아보인다. 하지만?

## 7.1.3 명시적 분기 (Explicit forking)

#### map2의 두 인수를 병렬로 평가하는 것이 항상 바람직할까?

In [ ]:
Par.map2(Par.unit(1), Par.unit(1))(_ + _)

#### 분기를 더 명시적으로 만들면 어떨까?

In [ ]:
def sum(ints: IndexedSeq[Int]) : Par[Int] =
    if (ints.size <= 1)
        Par.unit(ints.headOptoin getOrElse 0)
    else {
        Val (l, r) = ints.splitAt(ints.length/2)
        Par.map2(Par.fork(sum(l)), Par.fork(sum(r)))(_ + _)
    }

In [ ]:
def unit[A](a: A) : Par[A]
def lazyUnit[A](a: => A) : Par[A] = fork(unit(a))

#### get함수의 이름을 run으로 바꾸자.

In [ ]:
def run[A](a: Par[A]) : A

# 7.2 표현의 선택 (Picking a representation)

<img src="figures/list7.4.png" width=600 >  

<img src="figures/cap7.4.png" width=600 >  

In [ ]:
def run[A](s : ExecutorSerivce)(a : Par[A]) : A

In [ ]:
type Par[A] = ExecutorService => Future[A]

In [ ]:
def run[A](s : ExecutorService)(a : Par[A]) : Future[A] = a(s)

# 7.3 API의 정련 (Refining the API)

# 7.4 API의 대수 (The algebra of an API)

* 7.4.1 map에 관한 법칙 (The law of mapping)
* 7.4.2 for에 관한 법칙 (The law of forking)
* 7.4.3 법칙 깨기: 미묘한 버그 하나 (Breaking the law: a subtle bug)
* 7.4.4 행위자를 이용한 완전 비차단 Par 구현 (A fully non-blocking Par implementation using actors)

## 7.4.1 map에 관한 법칙 (The law of mapping)

## 7.4.2 for에 관한 법칙 (The law of forking)

## 7.4.3 법칙 깨기: 미묘한 버그 하나 (Breaking the law: a subtle bug)

## 7.4.4 행위자를 이용한 완전 비차단 Par 구현 (A fully non-blocking Par implementation using actors)

# 7.5 조합기들을 가장 일반적인 형태로 정련 (Refining combiators to their most general form)

# 7.6 요약 (Summary)

<img src="figures/cap7.10.png" width=600 >  

In [ ]:
<img src="figures/cap7.6.png" width=600 >     

In [ ]:
<img src="figures/cap7.11.png" width=600 >    

In [ ]:
<img src="figures/cap7.7.png" width=600 >     

In [ ]:
<img src="figures/list7.5-1.png" width=600 >  

In [ ]:
<img src="figures/cap7.12.png" width=600 >    

In [ ]:
<img src="figures/cap7.8.png" width=600 >     

In [ ]:
<img src="figures/list7.5-2.png" width=600 >  

In [ ]:
<img src="figures/cap7.9.png" width=600 >     

In [ ]:
<img src="figures/list7.6.png" width=600 >  

In [ ]:
<img src="figures/list7.7.png" width=600 >  

In [ ]:
<img src="figures/cap7.5.png" width=600 >     